<a href="https://colab.research.google.com/github/crystalclcm/JobPostings/blob/main/8_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data cleaning lab

##Exercise 1

The `unclean_sales.csv` dataset from todays Lecture Examples has multiple issues that need to be cleaned. Some were covered in class, and the rest are still there for you to work on

###Missing values
Age, PurchaseAmount, Name, City, SignupDate, and ProductCategory include NaN, blanks, and placeholders like "N/A".

###Duplicates
Two exact duplicates (Eve with OrderID A104 and Zane with OrderID A126) plus a duplicate CustomerID pattern (ID 5 repeated).

###Invalid records
Negative age (-5), extreme age (120), negative PurchaseAmount (-50), zero purchases (0 and 0.0), impossible dates (2021-13-01, 2021-02-30, 2021-06-31).

###Inconsistent data types
PurchaseAmount mixes integers, floats, and strings ("200", "N/A"). Age includes strings ("thirty"). Response mixes Y/N, y/n, "Yes", "No", "N/A".

###Inconsistent text/whitespace/case
Cities with different casing, spacing, and hyphenation ("new york", "New York", " SAN FRANCISCO", "Los Angeles", "Los-Angeles", "BOSTON", "Miami "). Product categories with trailing spaces and typos ("electronics ", "Electrnics", " Health").

###Date format chaos
ISO (YYYY-MM-DD), slashes (YYYY/MM/DD), US style (MM-DD-YYYY), blanks, and invalid calendar dates


In [2]:
import io
import pandas as pd


In [4]:
from google.colab import files
uploaded = files.upload()   # Choose your unclean_sales.csv


Saving unclean_sales.csv to unclean_sales (1).csv


In [11]:

import pandas as pd

# Using default options here (no special parsing yet) because real-world data can be messy
# and forcing types too early can hide problems. We want to "inspect before we fix".

df = pd.read_csv('/content/unclean_sales (1).csv')  # use the actual filename
# A quick look at the first few rows to get a sense of the columns and values.
df.head()


,CustomerID,OrderID,Name,Age,City,SignupDate,PurchaseAmount,Response,ProductCategory
0,3,A102,Charlie,NaN,Los Angeles,2021/03/20,150.0,Y,Home
1,17,A116,Peggy,41,Los Angeles,2021-02-30,155.0,No,Electrnics
2,2,A101,Bob,30,new york,2021-02-15,200.0,N,electronics
3,9,A108,Ivan,120,Boston,2021-08-20,180.0,N,Health
4,11,A110,Karl,NaN,Seattle,2021-10-30,210.0,Y,Health


In [14]:
print(df.shape)
 # Basic shape: tells us how many rows and columns we have (rows, columns).

print(df.info())
# info(): shows each column's data type and how many non-null values it has.
# This helps us see where missing data exists and whether columns came in as strings or numbers.

print(df.describe(include='all'))
# describe(include='all'): quick summary of values for each column.
# For numeric columns we get count/mean/min/max; for text we get unique/top/freq.
# This can reveal suspicious values (e.g., negative amounts, extreme ages, typos).

print(df.isnull().sum())
# Count missing values (NaN) by column. This outlines where we need to fill or leave as missing.

print(df.duplicated().sum())
# Count fully duplicated rows. Duplicates can be introduced by merges or repeated import

print(df.duplicated(subset=['OrderID']).sum())
# Count duplicates on a key column (OrderID). If the same order appears twice,
# we likely need to drop the duplicate to avoid double counting.



(29, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerID       29 non-null     int64  
 1   OrderID          29 non-null     object 
 2   Name             28 non-null     object 
 3   Age              26 non-null     object 
 4   City             27 non-null     object 
 5   SignupDate       28 non-null     object 
 6   PurchaseAmount   26 non-null     float64
 7   Response         27 non-null     object 
 8   ProductCategory  28 non-null     object 
dtypes: float64(1), int64(1), object(7)
memory usage: 2.2+ KB
None
        CustomerID OrderID  Name  Age    City  SignupDate  PurchaseAmount  \
count    29.000000      29    28   26      27          28       26.000000   
unique         NaN      27    26   20      20          26             NaN   
top            NaN    A104  Zane   29  Austin  2021-12-12             NaN   
freq     

In [20]:

df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
# Convert Age to numeric.
# errors='coerce' means any non-numeric value (e.g., 'thirty') becomes NaN instead of raising an error.
# This is a safe conversion to avoid breaking the notebook when data is messy.

df['PurchaseAmount'] = pd.to_numeric(df['PurchaseAmount'], errors='coerce')
#Convert PurchaseAmount to numeric.
# If the dataset had currency symbols or stray text, we'd strip them first,
# but this "simple" version just attempts a direct conversion.


# Fix text fields
df['City'] = df['City'].str.strip().str.title()
# Clean text fields: remove extra spaces and standardize case so categories match.
# City: strip leading/trailing spaces and convert to Title Case (e.g., 'new york' -> 'New York').

df['ProductCategory'] = df['ProductCategory'].str.strip().str.title()
# ProductCategory: same approach. This reduces accidental duplicates like 'health' vs 'Health'.

df['Response'] = df['Response'].str.strip().str.upper()
# Response: standardize to uppercase so 'y', 'Y', 'yes' all eventually align to 'Y' or 'N'.
# In the simple version, we only uppercase; if needed, we can map variants to 'Y'/'N'


# Fix dates (replace / with - only)
df['SignupDate'] = df['SignupDate'].str.replace('/', '-')
# Dates: replace slashes with hyphens to improve consistency.
# We keep this simple (no complex regex). If your lecturer expects parsing, we’ll do that below.


# Remove invalid values
df.loc[df['Age'] < 0, 'Age'] = None
df.loc[df['Age'] > 110, 'Age'] = None
# Remove invalid numeric values as a quality assurance step.
# Age: negative or extreme values are set to missing (NaN) — a "judgement call" per the lecture.

df.loc[df['PurchaseAmount'] <= 0, 'PurchaseAmount'] = None
# PurchaseAmount: values <= 0 are suspicious; we set them to missing.
# Depending on your business rules, a zero might mean a refund. Here we choose to impute later.


# Fill missing values
df['Age'] = df['Age'].fillna(df['Age'].median())
df['PurchaseAmount'] = df['PurchaseAmount'].fillna(df['PurchaseAmount'].median())
# Fill missing values with simple, defensible choices:
# Numeric columns: median (robust to outliers compared to mean).

df['City'] = df['City'].fillna('Unknown')
df['Name'] = df['Name'].fillna('Unknown')
df['ProductCategory'] = df['ProductCategory'].fillna('Unknown')
df['Response'] = df['Response'].fillna('Unknown')
# Text columns: fill with 'Unknown' where data is missing.
# This prevents dropping many rows and keeps analysis possible.


# Remove duplicates
df = df.drop_duplicates()
# Remove duplicates to prevent double counting in analysis.
# First remove exact duplicates across all columns.

df = df.drop_duplicates(subset=['OrderID'])# Then ensure each OrderID is unique. If the business rule says one row per order, this is necessary



In [18]:
df.to_csv('/content/clean_sales.csv', index=False)

In [19]:

print(df.shape)                 # rows, columns after cleaning
print(df.info())                # types and non-null counts
print(df.isnull().sum())        # any remaining missing values?
print(df.duplicated().sum())    # any full duplicates left?
print(df.duplicated(subset=['OrderID']).sum())  # duplicates on key?
print(df.describe(include='all'))               # quick summary


(27, 9)
<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, 0 to 28
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerID       27 non-null     int64  
 1   OrderID          27 non-null     object 
 2   Name             27 non-null     object 
 3   Age              27 non-null     float64
 4   City             27 non-null     object 
 5   SignupDate       26 non-null     object 
 6   PurchaseAmount   27 non-null     float64
 7   Response         27 non-null     object 
 8   ProductCategory  27 non-null     object 
dtypes: float64(2), int64(1), object(6)
memory usage: 2.1+ KB
None
CustomerID         0
OrderID            0
Name               0
Age                0
City               0
SignupDate         1
PurchaseAmount     0
Response           0
ProductCategory    0
dtype: int64
0
0
        CustomerID OrderID     Name        Age     City  SignupDate  \
count    27.000000      27       27  27.00000

## Exercise 2
Choose a data cleaning dataset on Kaggle.
https://www.kaggle.com/datasets?search=data+cleaning&fileType=csv

Don't just blindly follow the actions from class: at each stage think about the data and what is appropriate.
1. Import
2. Explore
3. Missing values
4. Duplicates
5. Invalid and inconsistent values

In [ ]:
# Add your code here